In [1]:
"""
Assignment 2, Task 3: Object Tracking (Full Comparison)
- Compares KCF and CSRT trackers.
- Asks user to select ONE initial bounding box.
- Runs both trackers sequentially on the same video with the same box.
- Displays the output video for each tracker.
- Prints a final comparison report.
"""


'\nAssignment 2, Task 3: Object Tracking (Full Comparison)\n- Compares KCF and CSRT trackers.\n- Asks user to select ONE initial bounding box.\n- Runs both trackers sequentially on the same video with the same box.\n- Displays the output video for each tracker.\n- Prints a final comparison report.\n'

In [2]:
import cv2
import sys
import time
import pandas as pd
import os
import numpy as np


In [4]:
# !!! --- SET YOUR VIDEO FILE PATHS HERE --- !!!
VIDEO_PATH = "test.gif"  # A short .gif video for tracking
# !!! --------------------------------- !!!

def run_tracking_comparison(video_path):
    
    # --- 1. Load Video and Select ROI ---
    video = cv2.VideoCapture(video_path)
    if not video.isOpened():
        print(f"Error: Could not open video file: {video_path}")
        return

    ret, frame = video.read()
    if not ret:
        print("Error: Could not read first frame.")
        video.release()
        return

    print("Select the object to track and press 'Enter' or 'Space'.")
    print("Press 'c' to cancel selection.")
    
    # Use cv2.selectROI for interactive box selection
    bbox = cv2.selectROI("Select Object", frame, True, False)
    cv2.destroyWindow("Select Object")
    
    if not bbox or bbox[2] == 0 or bbox[3] == 0:
        print("No bounding box selected. Exiting.")
        video.release()
        return
        
    print(f"Initial Bounding Box selected: {bbox}")

    # --- 2. Define Trackers to Compare ---
    def get_tracker_constructor(name):
        """
        Return a tracker constructor for the given tracker name.
        Tries several locations to support different OpenCV builds.
        """
        # Try cv2.legacy (newer opencv-contrib builds)
        if hasattr(cv2, "legacy"):
            legacy = getattr(cv2, "legacy")
            ctor = getattr(legacy, f"Tracker{name}_create", None)
            if callable(ctor):
                return ctor
        # Try top-level in cv2 (some builds expose them here)
        ctor = getattr(cv2, f"Tracker{name}_create", None)
        if callable(ctor):
            return ctor
        # Try old generic factory function
        ctor_generic = getattr(cv2, "Tracker_create", None)
        if callable(ctor_generic):
            return lambda: ctor_generic(name)
        raise AttributeError(f"No constructor found for tracker '{name}' in cv2")

    # Build tracker constructors only for trackers available in this OpenCV build.
    # Try several common tracker names and skip those not present to avoid AttributeError.
    trackers_to_try = ["KCF", "CSRT", "MOSSE", "MIL", "TLD", "MEDIANFLOW"]
    trackers_to_compare = {}
    for name in trackers_to_try:
        try:
            ctor = get_tracker_constructor(name)
            trackers_to_compare[name] = ctor
        except AttributeError as e:
            # Print a warning and continue if a tracker constructor is not available.
            print(f"Warning: {e}. Skipping tracker '{name}'.")
    
    results = []

    # --- 3. Run Each Tracker Sequentially ---
    for tracker_name, tracker_constructor in trackers_to_compare.items():
        
        print(f"\n--- Running {tracker_name} Tracker ---")
        
        # Reset video to the beginning
        video.set(cv2.CAP_PROP_POS_FRAMES, 0)
        ret, frame = video.read() # Read first frame again
        
        # Initialize the tracker
        tracker = tracker_constructor()
        try:
            tracker.init(frame, bbox)
        except Exception as e:
            print(f"Error initializing {tracker_name}: {e}. Skipping.")
            continue
            
        frame_count = 0
        total_fps = 0
        success_count = 0
        
        print("Tracking in progress... Press 'q' to skip to the next tracker.")
        
        while True:
            ret, frame = video.read()
            if not ret:
                break
                
            frame_count += 1
            start_time = time.time()
            
            # Update the tracker
            success, new_bbox = tracker.update(frame)
            
            # Calculate FPS safely (avoid division by zero)
            elapsed = time.time() - start_time
            if elapsed <= 0:
                fps = 0.0
            else:
                fps = 1.0 / elapsed
            total_fps += fps
            
            if success:
                success_count += 1
                (x, y, w, h) = [int(v) for v in new_bbox]
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                status_text = "Success"
                color = (0, 255, 0)
            else:
                status_text = "Tracking Failure"
                color = (0, 0, 255)

            # Display info on frame
            cv2.putText(frame, tracker_name, (10, 20), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)
            cv2.putText(frame, status_text, (10, 40), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
            cv2.putText(frame, f"FPS: {int(fps)}", (10, 60), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 2)
            
            cv2.imshow(f"Tracking with {tracker_name}", frame)
            
            if cv2.waitKey(1) & 0xFF == ord('q'):
                print("Skipping to next tracker...")
                break
        
        # Calculate final stats for this tracker
        avg_fps = total_fps / frame_count if frame_count > 0 else 0
        success_rate = (success_count / frame_count) * 100 if frame_count > 0 else 0
        
        results.append({
            "Tracker": tracker_name,
            "Avg. FPS": f"{avg_fps:.2f}",
            "Success Rate": f"{success_rate:.2f}%",
            "Total Frames": frame_count,
            "Failed Frames": frame_count - success_count
        })
        
        cv2.destroyWindow(f"Tracking with {tracker_name}")

    video.release()
    cv2.destroyAllWindows()

    # --- 4. Final Report ---
    print("\n\n" + "="*50)
    print("      ASSIGNMENT 3 - FINAL COMPARISON REPORT")
    print("="*50)
    
    if results:
        report_df = pd.DataFrame(results)
        print(report_df.to_string(index=False))
    else:
        print("No tracking results to display.")

if __name__ == "__main__":
    if not os.path.exists(VIDEO_PATH):
        print(f"Error: Video file not found at '{VIDEO_PATH}'.")
        print("Please edit the script to set the correct VIDEO_PATH.")
    else:
        run_tracking_comparison(VIDEO_PATH)

Select the object to track and press 'Enter' or 'Space'.
Press 'c' to cancel selection.
Initial Bounding Box selected: (78, 143, 96, 127)

--- Running KCF Tracker ---
Tracking in progress... Press 'q' to skip to the next tracker.

--- Running CSRT Tracker ---
Tracking in progress... Press 'q' to skip to the next tracker.

--- Running MOSSE Tracker ---
Tracking in progress... Press 'q' to skip to the next tracker.

--- Running MIL Tracker ---
Tracking in progress... Press 'q' to skip to the next tracker.

--- Running TLD Tracker ---
Tracking in progress... Press 'q' to skip to the next tracker.


      ASSIGNMENT 3 - FINAL COMPARISON REPORT
Tracker Avg. FPS Success Rate  Total Frames  Failed Frames
    KCF   156.93      100.00%            40              0
   CSRT    27.34      100.00%            40              0
  MOSSE   837.41      100.00%            40              0
    MIL    23.99      100.00%            40              0
    TLD    85.68      100.00%            40              

CF, CSRT, MOSSE, MIL, and TLD—were perfectly successful on this test. They all achieved a 100.00% success rate and had zero failed frames over the 40 frames tested. The main difference between them is clearly their speed, shown by the Average FPS. The MOSSE tracker is by far the fastest, with an extremely high performance of 837.41 FPS. KCF (156.93 FPS) and TLD (85.68 FPS) are also very fast, but CSRT (27.34 FPS) and MIL (23.99 FPS) are significantly slower in comparison. So, while all of them worked perfectly in terms of accuracy, MOSSE is the clear winner in terms of processing speed.